In [1]:
from premethod import *
from pegasus_fine_tune import *
from generate_pegasus import *
from combine import *
from rank import rank
from rank_training import rank_train
from p_at_1 import p_at_k
from keybart_finetune import *
from keybart_generate import get_pred_Keybart
from bart_finetune import fine_tune_bart
from bart_generate import get_pred_bart
### 

[nltk_data] Downloading package stopwords to /home/delab/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
2022-10-11 11:10:46,628 - sentence_transformers.cross_encoder.CrossEncoder - INFO - Use pytorch device: cuda
[nltk_data] Downloading package stopwords to /home/delab/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [7]:
datadir = ['../dataset/EUR-Lex/','../dataset/Wiki500K/']
k_fold = [0,1,2,3,4]
tasks = ['test','train','valid']
models = {'pega':0,'bart':1,'kb':0}
gener = "generate_result/"

In [ ]:
dataname=[datadir[1]+tasks[0]+"_labels20.txt",datadir[1]+tasks[1]+"_labels20.txt"]
get_all_labels(dataname,datadir[1]+"all_labels20.txt")
get_all_stemlabels(datadir[1]+'all_labels20.txt',datadir[1]+'all_stemlabels20.txt')

In [ ]:
# pegasus
if models['pega']:
        Pegasus_fine_tune(datadir[0],tasks[1]+'_finetune.json',tasks[0]+'_finetune.json',"pegasus_save","pegasus_check")
        for i in range(2):
            get_pred_Pegasus_fast(datadir[1],gener+tasks[i]+"_pred.txt",tasks[i]+"_finetune.json","pegasus_save")
            get_combine_list(datadir[1],gener+tasks[i]+"_pred.txt","all_stemlabels.txt",tasks[i]+"_combine_labels.txt")
        rank_train(datadir[1],tasks[1]+"_texts.txt",tasks[1]+"_combine_labels.txt",tasks[1]+"_labels_stem.txt","cr_en")
        rank(datadir[1],tasks[0]+"_texts.txt",tasks[0]+"_combine_labels.txt","cr_en",tasks[0]+"_ranked_labels.txt")
        res = p_at_k(datadir[1],tasks[0]+"_labels_stem.txt",tasks[0]+"_ranked_labels.txt",datadir[1]+"res_pega.txt")


In [11]:
if models['bart']:
        #fine_tune_bart(datadir[1],tasks[1]+'_finetune20.json',tasks[0]+'_finetune20.json','bart_save','bart_check')
        for i in range(2):
            get_pred_bart(datadir[1],gener+tasks[i]+"_pred_ba.txt",tasks[i]+"_finetune20.json","bart_save")
            bart_clean(datadir[1]+gener+tasks[i]+"_pred_ba.txt",datadir[1]+gener+tasks[i]+"_pred_ba_c.txt")
            get_combine_list(datadir[1],gener+tasks[i]+"_pred_ba.txt","all_labels20.txt",gener+tasks[i]+"_combine_labels_ba.txt")
        rank_train(datadir[1],tasks[1]+"_texts.txt",gener+tasks[1]+"_combine_labels_ba.txt",tasks[1]+"_labels20.txt","cr_en_ba")
        rank(datadir[1],tasks[0]+"_texts20.txt",gener+tasks[0]+"_combine_labels_ba.txt","cr_en_ba",gener+tasks[0]+"_ranked_labels_ba.txt")
        res = p_at_k(datadir[1],tasks[0]+"_labels20.txt",gener+tasks[0]+"_ranked_labels_ba.txt",datadir[1]+"ba_res.txt")

loading configuration file ../dataset/Wiki500K/bart_save/config.json
Model config BartConfig {
  "_name_or_path": "facebook/bart-base",
  "activation_dropout": 0.1,
  "activation_function": "gelu",
  "add_bias_logits": false,
  "add_final_layer_norm": false,
  "architectures": [
    "BartForConditionalGeneration"
  ],
  "attention_dropout": 0.1,
  "bos_token_id": 0,
  "classif_dropout": 0.1,
  "classifier_dropout": 0.0,
  "d_model": 768,
  "decoder_attention_heads": 12,
  "decoder_ffn_dim": 3072,
  "decoder_layerdrop": 0.0,
  "decoder_layers": 6,
  "decoder_start_token_id": 2,
  "dropout": 0.1,
  "early_stopping": true,
  "encoder_attention_heads": 12,
  "encoder_ffn_dim": 3072,
  "encoder_layerdrop": 0.0,
  "encoder_layers": 6,
  "eos_token_id": 2,
  "forced_bos_token_id": 0,
  "forced_eos_token_id": 2,
  "gradient_checkpointing": false,
  "id2label": {
    "0": "LABEL_0",
    "1": "LABEL_1",
    "2": "LABEL_2"
  },
  "init_std": 0.02,
  "is_encoder_decoder": true,
  "label2id": {
   

output: ../dataset/Wiki500K/generate_result/test_pred_ba.txt
src_data: ../dataset/Wiki500K/test_finetune20.json
model_path: ../dataset/Wiki500K/bart_save


All model checkpoint weights were used when initializing BartForConditionalGeneration.

All the weights of BartForConditionalGeneration were initialized from the model checkpoint at ../dataset/Wiki500K/bart_save.
If your task is similar to the task the model of the checkpoint was trained on, you can already use BartForConditionalGeneration for predictions without further training.
Didn't find file ../dataset/Wiki500K/bart_save/added_tokens.json. We won't load it.
loading file ../dataset/Wiki500K/bart_save/vocab.json
loading file ../dataset/Wiki500K/bart_save/merges.txt
loading file None
loading file ../dataset/Wiki500K/bart_save/special_tokens_map.json
loading file ../dataset/Wiki500K/bart_save/tokenizer_config.json
tokenizer config file saved in ../dataset/Wiki500K/bart_tokenizer/tokenizer_config.json
Special tokens file saved in ../dataset/Wiki500K/bart_tokenizer/special_tokens_map.json
  0%|          | 1/39188 [00:00<8:44:11,  1.25it/s]

Autism spectrum, Disability rights, Neurodevelopmental disorders, Syndromes, Transcription factor deficiencies, Tropical and subtropicaluddin II and III, Vaccination-related cutaneous conditions, World Health Organization essential medicines']======x80x93West Virginia


  0%|          | 99/39188 [00:53<5:53:04,  1.85it/s]


KeyboardInterrupt: 

In [ ]:
if models['kb']:    
        #kb_fine_tune(datadir[0],"kb_save","kb_check")
        fine_tune_keybart(datadir[1],tasks[1]+'_finetune.json',tasks[0]+'_finetune.json','keybart_save','keybart_test')
        for i in range(2):
            get_pred_Keybart(datadir[0],gener+tasks[i]+"_pred_kb.txt",tasks[i]+"_finetune.json","keybart_save")
            bart_clean(datadir[0]+gener+tasks[i]+"_pred_kb.txt",datadir[0]+gener+tasks[i]+"_pred_kb_c.txt")
            get_combine_list(datadir[0],gener+tasks[i]+"_pred_kb_c.txt","all_labels_sterm.txt",gener+tasks[i]+"_combine_labels_kb.txt")

##### K则验证

In [ ]:
for j in k_fold:
    k_dir = "/K_fold/"+"K_"+str(j)+"/"
    Pegasus_fine_tune(datadir[0]+k_dir,tasks[1]+'.json',tasks[0]+'.json',"pegasus_save","pegasus_check")
    for i in range(2):
        get_pred_Pegasus(datadir[0]+k_dir,tasks[i]+"_pred.txt",tasks[i]+".json","pegasus_save")
        get_combine_list(datadir[0]+k_dir,tasks[i]+"_pred.txt","all_stemlabels.txt",tasks[i]+"_combine_labels.txt")
    rank_train(datadir[0]+k_dir,tasks[1]+"_texts.txt",tasks[1]+"_combine_labels.txt",tasks[1]+"_labels_stem.txt","cr_en_"+str(j))
    rank(datadir[0]+k_dir,tasks[0]+"_texts.txt",tasks[0]+"_combine_labels.txt","cr_en_"+str(j),tasks[0]+"_ranked_labels.txt")
    res = p_at_k(datadir[0]+k_dir,tasks[0]+"_labels_stem.txt",tasks[0]+"_ranked_labels.txt",datadir[0]+k_dir+"res.txt")